# Text Preprocessing

I Prétraitement

- Détection des ponctuations déterminant les fins de phrases, segmentation des phrases
- Mise en minuscules
- Détection des symboles pertinents, enlèvement des ponctuations et symboles superflus
- Détection des espaces manquants
- Correction orthographique des mots

Un ensemble facultatif de traitements est le suivant :

- Enlèvement de stopwords
- Étiquetage des mots en pos
- Racinisation des mots

    
    
II Vectorisation

In [1]:
# path to anaconda : C:\ProgramData\Anaconda3

from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import json
import random
import itertools

In [2]:
import sys
sys.version

'3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]'

In [3]:
# for manipulating dataframes, arrays, containers...
import numpy as np
np.set_printoptions(threshold=np.nan)
import pandas as pd
import unidecode
from unidecode import unidecode

# equivalent of numpy for very large arrays (such as a database of pretrained word vectors)
# optimized for SQL-like operations, not for math computations
# see https://bcolz.readthedocs.io/en/latest/intro.html

# installer avec la commande : conda install bcolz
import bcolz

import pickle

In [4]:
# for viz
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [5]:
# for machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import euclidean_distances

In [6]:
# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

print(torch.__version__)

use_cuda = False
print(torch.cuda.is_available())
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

0.4.1
False


In [7]:
# for NLP
import gensim
from gensim.models import KeyedVectors

import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer

from textblob import TextBlob

In [8]:
import warnings
warnings.filterwarnings('ignore')

## Import du corpus

Import des données, issues de https://asrs.arc.nasa.gov/search/database.html

In [9]:
path_to_rep = 'C:\\Users\jbaujogue\Desktop\Scripts'

In [10]:
data = pd.read_csv(path_to_rep + '\data\Tickets_ASRS\ASRS_DBOnline_2018.csv', sep=',', header=1)
#Dates = ['2014_1', '2014_2', '2015_1', '2015_2', '2016_1', '2016_2', '2017_1', '2017_2']
#for date in Dates :
#    data_bis = pd.read_csv(path_to_rep + '\data\Tickets_ASRS\ASRS_DBOnline_' + date + '.csv', sep=',', header=1)
#    data = pd.concat([data, data_bis], ignore_index = True) #data.append(data_bis)
data.shape

(2400, 97)

In [11]:
data.head()

,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Relative Position.Angle.Radial,Relative Position.Distance.Nautical Miles,Altitude.AGL.Single Value,Altitude.MSL.Single Value,Flight Conditions,...,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Callback,Narrative.1,Callback.1,Synopsis,Unnamed: 96
0,1507557,201801,1201-1800,C90.TRACON,IL,NaN,NaN,NaN,5000.0,NaN,...,In-flight,Flight Crew Requested ATC Assistance / Clarifi...,Chart Or Publication; Airport; Company Policy;...,Company Policy,Aircraft X was assigned a runway approximately...,NaN,NaN,NaN,MLI Approach Controller reported ATC denied th...,NaN
1,1508049,201801,1201-1800,ZZZ.Airport,US,NaN,NaN,NaN,20000.0,VMC,...,In-flight,Air Traffic Control Provided Assistance; Fligh...,Aircraft; Human Factors,Aircraft,Climbing out of approximately FL200; we observ...,NaN,NaN,NaN,B737 Captain reported a fuel pump malfunction ...,NaN
2,1508078,201801,1201-1800,ZZZ.Airport,US,NaN,NaN,NaN,25000.0,NaN,...,In-flight,Flight Crew Overcame Equipment Problem; Genera...,Aircraft,Aircraft,Dual partially jammed control columns. Not sur...,Reporter stated he checked back with Maintenan...,During our initial descent from FL360 to FL240...,NaN,EMB-175 flight crew reported partially jammed ...,NaN
3,1508434,201801,0601-1200,ZZZ.Airport,US,NaN,NaN,0.0,NaN,VMC,...,In-flight,General Maintenance Action,Aircraft,Aircraft,On landing; Flight Crew felt moderate to sever...,NaN,The incident began with a cleared Maintenance ...,NaN,B737 flight crew reported a blown main tire on...,NaN
4,1508441,201801,1801-2400,ZZZ.Airport,US,NaN,NaN,NaN,1800.0,VMC,...,In-flight,Flight Crew Landed in Emergency Condition; Fli...,Aircraft,Aircraft,This flight was a ferry flight to reposition t...,NaN,NaN,NaN,A319 Captain reported malfunctioning landing g...,NaN


Sélection des attributs intéressants 

In [12]:
Important_Head = [0, 91, 95]
df = data.ix[ :50, Important_Head].copy()
df.columns = ['ACN', 'Description', 'Résumé']

In [13]:
df.to_csv(path_to_rep + '\data\Tickets_ASRS\descriptions_resumes_preprocessing_0.csv', sep=',', header=True, index = False)
# df = pd.read_csv(path_to_rep + '\data\Tickets_ASRS\descriptions_resumes_preprocessing_0.csv', sep=',', header=1)

In [62]:
def printExample(df, index) :
    ex_des = df.ix[index, 1]
    ex_res = df.ix[index, 2]
    print('Description :',ex_des, '\n \nRésumé :',ex_res)
    
printExample(df, 45)

Description : I was completing my second instructional flight of the day. This was a cross country flight meant to prepare my student for his upcoming check ride. We departed and proceeded to CHD; our first airport of landing. We called tower and received the following instruction: 'join left base runway 4L; report 4 miles.' We maneuvered for the base and at the appropriate distance made the requisite report. As we approached the base to final turn; my student was focused on accomplishing a procedural task. Not wanting to overshoot the centerline for fear of having to maneuver back to the runway; I told my student to stop his procedure and turn. I was also noting the TCAS showing traffic 12 o'clock and closing fast. As we turned the TCAS sounded its alert; by that point I already saw the traffic.My first thought was: 'this guy must be violating CHD's class delta.' I had not picked anything up over the frequency that he was there. Nonetheless the traffic was flying parallel to me 0.2 na

## Segmentation et normalisation de texte

- segmentation de texte
- normalisation (mise en minuscules, enlèvement des symboles et ponctuations)
- enlèvement des stopwords

In [15]:
# --------------------------- Normalisation -------------------------------
def unicodeToAscii(s):
    """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    '''Remove rare symbols from a string'''
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"[^a-zA-Z0-9]+", r" ", s)  # 'r' before a string is for 'raw' # ?&\%\_\- removed # set('''.,:;()*#&-_%!?/\'")''')
    return s



# -------------------- segmentation de texte -------------------------------------
def segmentText(text, stopwords = []) :
    '''Segments a string into a list of sentences, each consisting of a list of words as strings'''
    new_text = text.split('.')
    new_text = [normalizeString(line) for line in new_text]
    new_text = [[word for word in line.split() if word not in stopwords] for line in new_text]
    new_text = [line for line in new_text if line!= []]
    return new_text

    
def segmentTextNltk(text, stopwords = []) :
    '''Segments a string into a list of sentences, each consisting of a list of words as strings'''
    #text = clean_sentence_punct(text.lower())
    text = text.lower()
    new_text = sent_tokenize(text) 
    new_text = [word_tokenize(line) for line in new_text]
    new_text = [[word for word in line if word not in stopwords] for line in new_text]
    new_text = [line for line in new_text if line!= []]
    return new_text
    

# stopwords = nltk.corpus.stopwords.words('english')


def formatText1(df) :
    dfe = df.copy()
    for i in range(len(dfe)) :
        text = dfe.ix[i, 1]
        new_text = segmentText(text)
        dfe.set_value(i, 'Description', new_text)
        text = dfe.ix[i, 2]
        dfe.set_value(i, 'Résumé', normalizeString(text))
    return dfe


def cleanPrint(df, index) :
    text = df.ix[index, 1]
    chaine = ''
    for line in text :
        chaine += ' '.join(line) + '\n\n'
    print(chaine)

In [16]:
df1 = formatText1(df)
#df1.to_csv(path_to_rep + '\data\Tickets_ASRS\descriptions_resumes_preprocessing_1.csv', sep=',', header=True, index = False)
#df1 = pd.read_csv(path_to_rep + '\data\Tickets_ASRS\descriptions_resumes_preprocessing_1.csv', sep=',', header=1)

In [63]:
printExample(df1, 45)

Description : [['i', 'was', 'completing', 'my', 'second', 'instructional', 'flight', 'of', 'the', 'day'], ['this', 'was', 'a', 'cross', 'country', 'flight', 'meant', 'to', 'prepare', 'my', 'student', 'for', 'his', 'upcoming', 'check', 'ride'], ['we', 'departed', 'and', 'proceeded', 'to', 'chd', 'our', 'first', 'airport', 'of', 'landing'], ['we', 'called', 'tower', 'and', 'received', 'the', 'following', 'instruction', 'join', 'left', 'base', 'runway', '4l', 'report', '4', 'miles'], ['we', 'maneuvered', 'for', 'the', 'base', 'and', 'at', 'the', 'appropriate', 'distance', 'made', 'the', 'requisite', 'report'], ['as', 'we', 'approached', 'the', 'base', 'to', 'final', 'turn', 'my', 'student', 'was', 'focused', 'on', 'accomplishing', 'a', 'procedural', 'task'], ['not', 'wanting', 'to', 'overshoot', 'the', 'centerline', 'for', 'fear', 'of', 'having', 'to', 'maneuver', 'back', 'to', 'the', 'runway', 'i', 'told', 'my', 'student', 'to', 'stop', 'his', 'procedure', 'and', 'turn'], ['i', 'was', 'a

In [64]:
cleanPrint(df1, 45)

i was completing my second instructional flight of the day

this was a cross country flight meant to prepare my student for his upcoming check ride

we departed and proceeded to chd our first airport of landing

we called tower and received the following instruction join left base runway 4l report 4 miles

we maneuvered for the base and at the appropriate distance made the requisite report

as we approached the base to final turn my student was focused on accomplishing a procedural task

not wanting to overshoot the centerline for fear of having to maneuver back to the runway i told my student to stop his procedure and turn

i was also noting the tcas showing traffic 12 o clock and closing fast

as we turned the tcas sounded its alert by that point i already saw the traffic

my first thought was this guy must be violating chd s class delta

i had not picked anything up over the frequency that he was there

nonetheless the traffic was flying parallel to me 0

2 nautical miles away at my

## Correction de mots

In [22]:
from textblob import Word
w = Word('mentionned')
w.spellcheck()

[('mentioned', 1.0)]

In [23]:
#https://rustyonrampage.github.io/text-mining/2017/11/28/spelling-correction-with-python-and-nltk.html
#https://textblob.readthedocs.io/en/dev/quickstart.html#spelling-correction

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)


# we should introduce a list of technical words as exeptions, in the spirit of stopwords


def correctText(text) :
    texte = text.copy()
    for i, line in enumerate(texte):
        for j, word in enumerate(line) :
            texte[i][j] = str(TextBlob(reduce_lengthening(word)).correct())
    return texte
    

def formatText2(df) :
    dfe = df.copy()
    for i in range(len(dfe)) :
        text = dfe.ix[i, 1]
        new_text = correctText(text)
        dfe.set_value(i, 'Description', new_text)
    return dfe

In [24]:
df2 = formatText2(df1) # change le df !!
df1 = formatText1(df)

In [65]:
printExample(df2, 45)

Description : [['i', 'was', 'completing', 'my', 'second', 'instructions', 'flight', 'of', 'the', 'day'], ['this', 'was', 'a', 'cross', 'country', 'flight', 'meant', 'to', 'prepare', 'my', 'student', 'for', 'his', 'coming', 'check', 'ride'], ['we', 'departed', 'and', 'proceeded', 'to', 'cod', 'our', 'first', 'airport', 'of', 'landing'], ['we', 'called', 'tower', 'and', 'received', 'the', 'following', 'instruction', 'join', 'left', 'base', 'runaway', 'll', 'report', '4', 'miles'], ['we', 'maneuvered', 'for', 'the', 'base', 'and', 'at', 'the', 'appropriate', 'distance', 'made', 'the', 'requisite', 'report'], ['as', 'we', 'approached', 'the', 'base', 'to', 'final', 'turn', 'my', 'student', 'was', 'focused', 'on', 'accomplishing', 'a', 'procedure', 'task'], ['not', 'wanting', 'to', 'overshoes', 'the', 'centerline', 'for', 'fear', 'of', 'having', 'to', 'maneuver', 'back', 'to', 'the', 'runaway', 'i', 'told', 'my', 'student', 'to', 'stop', 'his', 'procedure', 'and', 'turn'], ['i', 'was', 'als

In [66]:
cleanPrint(df2, 45)

i was completing my second instructions flight of the day

this was a cross country flight meant to prepare my student for his coming check ride

we departed and proceeded to cod our first airport of landing

we called tower and received the following instruction join left base runaway ll report 4 miles

we maneuvered for the base and at the appropriate distance made the requisite report

as we approached the base to final turn my student was focused on accomplishing a procedure task

not wanting to overshoes the centerline for fear of having to maneuver back to the runaway i told my student to stop his procedure and turn

i was also noting the twas showing traffic 12 o clock and closing fast

as we turned the twas sounded its alert by that point i already saw the traffic

my first thought was this guy must be violating cod s class felt

i had not picked anything up over the frequency that he was there

nonetheless the traffic was flying parallel to me 0

2 nautical miles away at my at

## I.3 Racinisation de mots

In [27]:
def stemming(text):
    texte = text.copy()
    for i, line in enumerate(texte):
        for j, word in enumerate(line) :
            texte[i][j] = PorterStemmer().stem(word)
    return texte


def formatText3(df) :
    dfe = df.copy()
    for i in range(len(dfe)) :
        text = dfe.ix[i, 1]
        new_text = stemming(text)
        dfe.set_value(i, 'Description', new_text)
    return dfe

In [28]:
df3 = formatText3(df2) # Change le df!!!!!
df2 = formatText2(df1)
df1 = formatText1(df)

In [67]:
cleanPrint(df3, 45)

i wa complet my second instruct flight of the day

thi wa a cross countri flight meant to prepar my student for hi come check ride

we depart and proceed to cod our first airport of land

we call tower and receiv the follow instruct join left base runaway ll report 4 mile

we maneuv for the base and at the appropri distanc made the requisit report

as we approach the base to final turn my student wa focus on accomplish a procedur task

not want to oversho the centerlin for fear of have to maneuv back to the runaway i told my student to stop hi procedur and turn

i wa also note the twa show traffic 12 o clock and close fast

as we turn the twa sound it alert by that point i alreadi saw the traffic

my first thought wa thi guy must be violat cod s class felt

i had not pick anyth up over the frequenc that he wa there

nonetheless the traffic wa fli parallel to me 0

2 nautic mile away at my attitud

it look like there were two peopl insid hard at work fli their plane

i call cod tower an

## I.4 POS tagging de mots

In [30]:
def tagging(text):
    texte = text.copy()
    for i, line in enumerate(texte):
        texte[i] = nltk.pos_tag(line)
    return texte


def formatText4(df) :
    dfe = df.copy()
    for i in range(len(dfe)) :
        text = dfe.ix[i, 1]
        new_text = tagging(text)
        dfe.set_value(i, 'Description', new_text)
    return dfe

In [31]:
df4 = formatText4(df2)

In [32]:
printExample(df4, 8)

Description : [[('i', 'VB'), ('a', 'DT'), ('private', 'JJ'), ('pilot', 'NN'), ('was', 'VBD'), ('getting', 'VBG'), ('dual', 'JJ'), ('instruction', 'NN'), ('in', 'IN'), ('the', 'DT'), ('practice', 'NN'), ('area', 'NN'), ('west', 'NN'), ('of', 'IN'), ('semi', 'NN')], [('we', 'PRP'), ('had', 'VBD'), ('just', 'RB'), ('finished', 'VBN'), ('slow', 'JJ'), ('flight', 'NN'), ('including', 'VBG'), ('turns', 'NNS'), ('of', 'IN'), ('90', 'CD'), ('degrees', 'NNS'), ('albeit', 'IN'), ('with', 'IN'), ('a', 'DT'), ('nose', 'JJ'), ('high', 'JJ'), ('attitude', 'NN')], [('i', 'NN'), ('was', 'VBD'), ('recovering', 'VBG'), ('the', 'DT'), ('plane', 'NN'), ('to', 'TO'), ('straight', 'VB'), ('and', 'CC'), ('level', 'VB'), ('flight', 'NN'), ('in', 'IN'), ('a', 'DT'), ('cruise', 'NN'), ('configuration', 'NN'), ('for', 'IN'), ('the', 'DT'), ('next', 'JJ'), ('maneuver', 'NN')], [('when', 'WRB'), ('i', 'NN'), ('looked', 'VBD'), ('left', 'VBN'), ('to', 'TO'), ('begin', 'VB'), ('a', 'DT'), ('clearing', 'NN'), ('turn'

In [71]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

## II.1 Création de langage



In [36]:
SOS_token = 0
EOS_token = 1
UNK_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in sentence.split():
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))

In [34]:
def generateLanguages(df):
    descriptions = df.ix[:, 1].values
    resumes = df.ix[:, 2].values
    lang = Lang('descriptions')
    lang.addDescriptions(descriptions)
    lang.addDescriptions(resumes)
    print("Mots comptés : ", lang.n_words)
    return lang

In [37]:
lang1 = generateLanguages(df1)
lang2 = generateLanguages(df2) # with corrections
lang3 = generateLanguages(df3) # with corrections + stemming

Mots comptés :  2439
Mots comptés :  2347
Mots comptés :  1994


### Comparaison avec des langages word2vec/GloVe

In [38]:
# tiré de http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
# pour entrainer un word2vec avec gensim : https://github.com/kavgan/nlp-text-mining-working-examples/blob/master/word2vec/Word2Vec.ipynb

full_language = gensim.models.KeyedVectors.load_word2vec_format(path_to_rep + '\data\word_vectors\GoogleNews-vectors-negative300.bin', binary=True)
vocab_word2vec = list(full_language.vocab.keys())

In [39]:
#taken from https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76
def vocabGlove(name) :
    words = []
    path = path_to_rep + '/data/word_vectors/GloVe/' + name 
    with open(path + '.txt', 'rb') as f:
        for l in f:
            line = l.decode().split()
            word = line[0]
            words.append(word)
    return words

vocab_glove = vocabGlove('glove_840B_300d') # ('glove.6B.50d')

In [40]:
len(vocab_glove)

2196017

In [41]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))


def printComparaison(nom, vocab_lang, intersect, reste) :
    print('proportion de mots du langage appartenants à {}  {:.2f} % \nproportion de mots du langage ny appartenant pas       {:.2f} %'.format(nom, len(intersect)*100/len(vocab_lang),len(reste)*100/len(vocab_lang) ) )


def comparaison(lang) :
    vocab_lang = list(lang.word2index.keys())
    intersect_word2vec = intersection(vocab_word2vec, vocab_lang)
    reste_word2vec = np.setdiff1d(vocab_lang, intersect_word2vec)
    intersect_glove = intersection(vocab_glove, vocab_lang)
    reste_glove = np.setdiff1d(vocab_lang, intersect_glove)
    printComparaison('word2vec', vocab_lang, intersect_word2vec, reste_word2vec)
    printComparaison('glove   ', vocab_lang, intersect_glove, reste_glove)
    return intersect_word2vec, reste_word2vec, intersect_glove, reste_glove

In [42]:
intersect1_word2vec, reste1_word2vec, intersect1_glove, reste1_glove = comparaison(lang1)

proportion de mots du langage appartenants à word2vec  91.80 % 
proportion de mots du langage ny appartenant pas       8.20 %
proportion de mots du langage appartenants à glove     97.83 % 
proportion de mots du langage ny appartenant pas       2.17 %


In [43]:
intersect2_word2vec, reste2_word2vec, intersect2_glove, reste2_glove = comparaison(lang2)

proportion de mots du langage appartenants à word2vec  93.57 % 
proportion de mots du langage ny appartenant pas       6.43 %
proportion de mots du langage appartenants à glove     98.55 % 
proportion de mots du langage ny appartenant pas       1.45 %


In [44]:
intersect3_word2vec, reste3_word2vec, intersect3_glove, reste3_glove = comparaison(lang3)

proportion de mots du langage appartenants à word2vec  76.48 % 
proportion de mots du langage ny appartenant pas       23.52 %
proportion de mots du langage appartenants à glove     92.23 % 
proportion de mots du langage ny appartenant pas       7.77 %


In [45]:
print(len(reste2_glove))
reste2_glove

34


array(['172rg', '300er', 'autothrottles', 'b747', 'b767', 'b777', 'c172',
       'ce560xl', 'deconflicted', 'doobi2', 'egpws', 'eicas', 'fl090',
       'fl165', 'fl200', 'fl200s', 'fl220', 'fl230', 'fl270', 'fl360',
       'fl370', 'fl390', 'fl400', 'fliteview', 'gpws', 'interlines',
       'jetblast', 'nmac', 'qrh', 'rnav', 'roylott', 'tracon', 'zeekk2',
       'zny'], dtype='<U15')

**Inférence des espaces manquants**

In [46]:
def checkWhetherBroken(vocab, clean_vocab) :
    exit = {}
    for word in vocab :
        exit[word] = True if word in clean_vocab else False
    return exit

def checkMissingSpaces(word, clean_vocab) :
    for word2 in clean_vocab :
        if word.startswith(word2) :
            rest = word.replace(word2, '')
            if rest in clean_vocab :
                return word2 + ' ' + rest
    return word

checkMissingSpaces('unknownwithin', lang2.word2index.keys())

'unknown within'

**Corrélation entre mots vectorisés avec word2vec**

In [59]:
full_language.similarity('finished', 'finishing')

0.7856169944530547

In [61]:
full_language.similarity('plane', 'aircraft')

0.7360518639176368

**Détection des valeurs numériques complexes**

In [74]:
nltk.pos_tag(['12;000ft']) # CD : Numerical value

[('12;000ft', 'CD')]